# Overview Materi

Source: https://www.youtube.com/watch?v=LDRbO9a6XPU

In [ ]:
ecision tree adalah metode yang digunakan dalam analisis data maupun machine learning untuk membantu pengambilan keputusan dengan menyusun setiap opsi atau langkah ke dalam bentuk struktur pohon bercabang, mulai dari sebuah node akar, bertingkat ke node keputusan, dan berakhir di node daun yang mewakili hasil akhir atau prediksi.

Jelaskan secara singkat apa itu decision tree menurut pemahamanmu!

# Import Data & Libraries

In [ ]:
from __future__ import print_function

# label kolom
header = ["color", "diameter", "label"]

# data training
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# data testing
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# Fungsi Dasar

In [ ]:
# fungsi mencari apa saja unique value dari suatu kolom
df['nama_kolom'].unique() akan menghasilkan nilai unik dari kolom nama_kolom.

Untuk menghitung jumlah nilai unik, gunakan nunique(), misal df['nama_kolom'].nunique().

Untuk mengetahui frekuensi kemunculan tiap unik value, bisa digunakan value_counts() seperti df['nama_kolom'].value_counts()

# contoh penggunaan
import pandas as pd

df = pd.DataFrame({
    'kolom_a': [1, 2, 2, 3, 4, 4, 4, 5]
})

unique_values = df['kolom_a'].unique()
print(unique_values)

In [ ]:
# fungsi Menghitung jumlah unique value dari suatu kolom
def class_counts(rows):
    counts =
    for row in rows:
        label = row
        if label not in counts:
            counts[label] = 1
        else:
            counts[label] += 1
    return counts

# contoh penggunaan
data = [1, 2, 2, 3, 4, 4, 4, 5]
print(class_counts(data))

Ellipsis

In [ ]:
# fungsi pengecekan suatu value numerik atau bukan
def is_numeric(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

# contoh penggunaan
print(is_numeric(42))        # True
print(is_numeric(3.14))      # True
print(is_numeric("123"))     # True, karena bisa dikonversi ke float
print(is_numeric("abc"))     # False
print(is_numeric(None))      # False

In [ ]:
# kelas untuk merepresentasikan pertanyaan pada decision tree
class Question:

    # inisialisasi kolom dan nilai pertanyaan
    def __init__(self, column, value):
        self.column = column
        self.value = value

    # mengecek apakah contoh data sesuai dengan pertanyaan
    def match(self, example):
        val = example[self.column]
        if isinstance(val, (int, float)):
            return val >= self.value
        else:
            return val == self.value

    # menampilkan pertanyaan dalam format string yang mudah dibaca
    def __repr__(self):
        condition = ">=" if isinstance(self.value, (int, float)) else "=="
        return f"Is column[{self.column}] {condition} {self.value}?"

# contoh penggunaan 1
q = Question(1, 5)
example1 = [3, 7]
example2 = [4, 3]
print(q)              # Output: Is column[1] >= 5?
print(q.match(example1))  # Output: True
print(q.match(example2))  # Output: False

In [ ]:
# membagi dataset menjadi dua berdasarkan pertanyaan
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows


# contoh penggunaan
true_rows, false_rows = partition(training_data, Question(0, 'Red'))
print(true_rows)  # Output: [['Red', 1, 'Grape'], ['Red', 1, 'Grape']]
print(false_rows)  # Output:

**apa itu gini impurity?**
<br> gini impurity berfungsi mengukur tingkat ketidakmurnian atau ketidakteraturan pada sebuah simpul (node) dalam pohon

In [ ]:
# menghitung nilai Gini Impurity untuk sebuah dataset
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

# contoh penggunaan
gini(training_data) :: 0.639

**apa itu information gain?**
<br> information gain berfungsi mengukur seberapa efektif sebuah fitur dalam memisahkan data berdasarkan kelas-kelasnya

In [ ]:
# menghitung nilai Information Gain dari pemisahan dataset
def info_gain(left, right, current_uncertainty):
    gain = current_uncertainty
    p = float(len(left)) / (len(left) + len(right))
    gain -= p * gini(left)
    gain -= (1 - p) * gini(right)
    return gain

# contoh penggunaan
gain = info_gain(true_rows, false_rows, gini(training_data))
print(gain)  # Output: 0.0

In [ ]:
# mencari pertanyaan terbaik untuk membagi dataset berdasarkan information gain tertinggi
def find_best_split(rows):
    best_gain = 0  # Nilai information gain terbaik
    best_question = None  # Pertanyaan terbaik
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # Asumsi kolom terakhir adalah label

    for col in range(n_features):
        values = set([row[col] for row in rows])  # Nilai unik di kolom ini
        for val in values:
            question = Question(col, val)

            # Membagi dataset
            true_rows, false_rows = partition(rows, question)

            # Lewati split jika tidak membagi data
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Hitung information gain
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # Simpan split terbaik
            if gain > best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

# contoh penggunaan
training_data = [
    [3, 'Red', 'Apple'],
    [1, 'Green', 'Grape'],
    [2, 'Red', 'Grape'],
    [3, 'Green', 'Apple']
]
best_gain, best_question = find_best_split(training_data)
print('Best Gain:', best_gain)
print('Best Question:', best_question)

# Fungsi Decision Tree

In [ ]:
# merepresentasikan node daun (leaf) pada decision tree yang berisi hasil prediksi
class Leaf:


    # inisialisasi leaf dengan menghitung jumlah kemunculan tiap kelas
    def __init__(self, rows):
        self.predictions = class_counts([row[-1] for row in rows])

In [ ]:
# merepresentasikan node keputusan (decision node) yang berisi pertanyaan dan cabang
class Decision_Node:

    # inisialisasi node dengan pertanyaan, cabang benar, dan cabang salah
    class Decision_Node:
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch



In [ ]:
# membangun decision tree secara rekursif
def build_tree(rows):
    gain, question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)
    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    return Decision_Node(question, true_branch, false_branch)

In [ ]:
# mencetak struktur decision tree secara rekursif dalam format teks
def classify(row, node):
    # base case: jika sudah mencapai leaf
    if isinstance(node, Leaf):
        return node.predictions
    # menentukan apakah mengikuti cabang true atau cabang false
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

# contoh penggunaan
training_data = [
    [3, 'Red', 'Apple'],
    [1, 'Green', 'Grape'],
    [2, 'Red', 'Grape'],
    [3, 'Green', 'Apple']
]
tree = build_tree(training_data)
row = [2, 'Red']
print(classify(row, tree))  # Output: prediksi kelas dalam bentuk dictionary

In [ ]:
def classify(row, node):
    # base case: jika sudah mencapai leaf
    if isinstance(node, Leaf):
        return node.predictions
    # menentukan apakah mengikuti cabang true atau cabang false
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

# contoh penggunaan
training_data = [
    [3, 'Red', 'Apple'],
    [1, 'Green', 'Grape'],
    [2, 'Red', 'Grape'],
    [3, 'Green', 'Apple']
]
tree = build_tree(training_data)
row = [2, 'Red']
print(classify(row, tree))  # Output: prediksi kelas dalam bentuk dictionary

In [ ]:
# menampilkan prediksi pada leaf dalam format persentase
def print_leaf(counts):
    total = sum(counts.values())
    probs = {}
    for label in counts:
        probs[label] = f"{int(counts[label] / total * 100)}%"
    return probs

# contoh penggunaan
counts = {"Apple": 2, "Grape": 1}
print(print_leaf(counts))  # Output: {'Apple': '66%', 'Grape': '33%'}


# Predict Using Decision Tree

In [ ]:
# menguji decision tree dengan data uji dan membandingkan hasil prediksi dengan label asli
# testing_data: list of rows, setiap row berisi fitur dan label asli di kolom terakhir
for row in testing_data:
    # prediksi menggunakan decision tree
    prediction = classify(row[:-1], tree)  # row[:-1] = fitur saja
    # label asli
    actual = row[-1]
    # tampilkan hasil prediksi dan label asli
    print(f"Prediksi: {print_leaf(prediction)}, Label asli: {actual}")